In [1]:
try:
    import google.colab  # noqa: F401

    # specify the version of DataEval (==X.XX.X) for versions other than the latest
    %pip install -q dataeval[torch]
except Exception:
    pass

In [2]:
import numpy as np

from dataeval.detectors.ood.ae_torch import OOD_AE
from dataeval.utils.torch.datasets import MNIST
from dataeval.utils.torch.models import AE


In [3]:
# Load in the training MNIST dataset and use the first 2000
train_ds = MNIST(root="./data/", train=True, download=True, size=2000, unit_interval=True, channels="channels_first")

# Split out the images and labels
images, labels = train_ds.data, train_ds.targets
input_shape = images[0].shape

Files already downloaded and verified


In [4]:
OOD_detectors = [
    OOD_AE(AE(input_shape=input_shape)),
]

In [5]:
for detector in OOD_detectors:
    print(f"Training {detector.__class__.__name__}...")
    detector.fit(images, threshold_perc=99, epochs=23)

Training OOD_AE...


In [6]:
corruption = MNIST(
    root="./data",
    train=True,
    download=False,
    size=2000,
    unit_interval=True,
    channels="channels_first",
    corruption="translate",
)
corrupted_images = corruption.data

Files already downloaded and verified


In [7]:
[(type(detector).__name__, np.mean(detector.predict(images).is_ood)) for detector in OOD_detectors]

[('OOD_AE', 0.01)]

In [8]:
[(type(detector).__name__, np.mean(detector.predict(corrupted_images).is_ood)) for detector in OOD_detectors]

[('OOD_AE', 0.912)]